<a href="https://colab.research.google.com/github/rinrin528/AIFFEL-Exploration/blob/main/%5BE_01%5Drock_scissors_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import os, glob
def resize_images(img_path):
	images=glob.glob(img_path + "/*.jpg")  
    
	print(len(images), " images to be resized.")

    # 파일마다 모두 28x28 사이즈로 바꾸어 저장합니다.
	target_size=(28,28)
	for img in images:
		old_img=Image.open(img)
		new_img=old_img.resize(target_size,Image.ANTIALIAS)
		new_img.save(img, "JPEG")
    
	print(len(images), " images resized.")

import numpy as np

def load_data(img_path, number_of_data=8435):  # 가위바위보 이미지 개수 총합에 주의하세요.
    # 가위 : 0, 바위 : 1, 보 : 2
    img_size=28
    color=3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)

    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1

    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1  
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_train)의 이미지 개수는", idx,"입니다.")
    return imgs, labels

In [ ]:
# train image 사이즈 변경
for i in ['scissor','rock','paper']:
    image_dir_path = os.getenv("HOME") + "/aiffel/enode1/rock_scissor_paper/"+i
    resize_images(image_dir_path)

2743  images to be resized.
2743  images resized.
2940  images to be resized.
2940  images resized.
2752  images to be resized.
2752  images resized.


## 훈련 데이터

In [ ]:
image_dir_path = os.getenv("HOME") + "/aiffel/enode1/rock_scissor_paper"
(x_train, y_train)=load_data(image_dir_path)
x_train_norm = x_train/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_train shape: {}".format(x_train.shape))
print("y_train shape: {}".format(y_train.shape))

학습데이터(x_train)의 이미지 개수는 8435 입니다.
x_train shape: (8435, 28, 28, 3)
y_train shape: (8435,)


## 네트워크 설계하기

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# model을 직접 만들어 보세요.
# Hint! model의 입력/출력부에 특히 유의해 주세요. 가위바위보 데이터셋은 MNIST 데이터셋과 어떤 점이 달라졌나요?
channel_1 = 32
channel_2 = 32
channel_3 = 64
n_dense = 34
n_train_epoch = 10

model=keras.models.Sequential()
model.add(keras.layers.Conv2D(channel_1,(3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(channel_2,(3,3), activation='relu'))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(channel_3,(3,3), activation='relu'))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(n_dense, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(3,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          18496     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 34)                1

## 모델 학습

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
history = model.fit(x_train_norm, y_train, epochs=n_train_epoch)

Epoch 1/10
264/264 [==============================] - 12s 22ms/step - loss: 1.0515 - accuracy: 0.4033
Epoch 2/10
264/264 [==============================] - 1s 4ms/step - loss: 0.7258 - accuracy: 0.6775
Epoch 3/10
264/264 [==============================] - 1s 3ms/step - loss: 0.4940 - accuracy: 0.7943
Epoch 4/10
264/264 [==============================] - 1s 3ms/step - loss: 0.3602 - accuracy: 0.8553
Epoch 5/10
264/264 [==============================] - 1s 3ms/step - loss: 0.2987 - accuracy: 0.8756
Epoch 6/10
264/264 [==============================] - 1s 3ms/step - loss: 0.2305 - accuracy: 0.9046
Epoch 7/10
264/264 [==============================] - 1s 3ms/step - loss: 0.1994 - accuracy: 0.9099
Epoch 8/10
264/264 [==============================] - 1s 3ms/step - loss: 0.1629 - accuracy: 0.9244
Epoch 9/10
264/264 [==============================] - 1s 4ms/step - loss: 0.1474 - accuracy: 0.9336
Epoch 10/10
264/264 [==============================] - 1s 3ms/step - loss: 0.1327 - accuracy: 0.93

## 테스트 데이터

In [ ]:
for i in ['rock','scissor','paper']:
	image_dir_path = os.getenv("HOME") + "/aiffel/enode1/rock_scissor_paper/test/"+i
	resize_images(image_dir_path)
test_image_dir_path = os.getenv("HOME") + "/aiffel/enode1/rock_scissor_paper/test"
(x_test, y_test)=load_data(test_image_dir_path, 428)
x_test_norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화

print("x_test shape: {}".format(x_test_norm.shape))
print("y_test shape: {}".format(y_test.shape))

109  images to be resized.
109  images resized.
210  images to be resized.
210  images resized.
109  images to be resized.
109  images resized.
학습데이터(x_train)의 이미지 개수는 428 입니다.
x_test shape: (428, 28, 28, 3)
y_test shape: (428,)


## 모델 평가

In [ ]:
test_loss, test_accuracy = model.evaluate(x_test_norm, y_test, verbose=2)
print("test_loss: {} ".format(test_loss))
print("test_accuracy: {}".format(test_accuracy))

14/14 - 3s - loss: 0.3323 - accuracy: 0.7617
test_loss: 0.3323216736316681 
test_accuracy: 0.7616822719573975


In [ ]:
predicted_result = model.predict(x_test_norm)  # model이 추론한 확률값. 
predicted_labels = np.argmax(predicted_result, axis=1)

# 실제 모델이 추론한 결과를 보자
index = {'가위':(0,100), '바위':(130,300), '보':(330,400)}
for idx in range(0,109):
    print('model.predict() 결과 : ', predicted_result[idx])
    print(f'model이 추론한 답 : {predicted_labels[idx]} 실제 데이터의 라벨 : {y_test[idx]}')

model.predict() 결과 :  [0.394366   0.32219312 0.2834409 ]
model이 추론한 답 : 0 실제 데이터의 라벨 : 0
model.predict() 결과 :  [9.9986756e-01 5.0860643e-07 1.3191983e-04]
model이 추론한 답 : 0 실제 데이터의 라벨 : 0
model.predict() 결과 :  [9.9966741e-01 1.3322267e-06 3.3125511e-04]
model이 추론한 답 : 0 실제 데이터의 라벨 : 0
model.predict() 결과 :  [0.32649416 0.32242477 0.35108113]
model이 추론한 답 : 2 실제 데이터의 라벨 : 0
model.predict() 결과 :  [0.40020832 0.5987885  0.00100321]
model이 추론한 답 : 1 실제 데이터의 라벨 : 0
model.predict() 결과 :  [0.303917   0.33632737 0.3597556 ]
model이 추론한 답 : 2 실제 데이터의 라벨 : 0
model.predict() 결과 :  [0.33292463 0.34888965 0.31818566]
model이 추론한 답 : 1 실제 데이터의 라벨 : 0
model.predict() 결과 :  [9.9839467e-01 7.5176038e-04 8.5358339e-04]
model이 추론한 답 : 0 실제 데이터의 라벨 : 0
model.predict() 결과 :  [9.9938917e-01 1.4169264e-04 4.6917971e-04]
model이 추론한 답 : 0 실제 데이터의 라벨 : 0
model.predict() 결과 :  [0.29447553 0.3639616  0.34156287]
model이 추론한 답 : 1 실제 데이터의 라벨 : 0
model.predict() 결과 :  [8.7264997e-01 1.5643639e-04 1.2719360e-01]
model이 추

In [ ]:
# try2로 데이터 양 보강한 후
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted_labels,target_names=['가위','바위','보']))

              precision    recall  f1-score   support

          가위       0.95      0.55      0.70       210
          바위       0.81      0.98      0.89       109
           보       0.59      0.94      0.73       109

    accuracy                           0.76       428
   macro avg       0.78      0.83      0.77       428
weighted avg       0.82      0.76      0.75       428



# 정확도 향상을 위한 시행착오
### 첫번째 문제상황
- 가위 데이터가 적은 편
- 대부분의 데이터를 보자기라고 판단
- 과적합되어서 훈련 데이터 정확도는 99% (epochs 늘리면 100% 이기도)

### try1) 모델층 쌓기

1. 원래 사용했던 모델
   - Conv2D(16, (3,3), relu)
   - MaxPool2D(2,2)
   - Conv2D(32, (3,3), relu)
   - MaxPool2D(2,2)
   - Conv2D(64, (3,3), relu)
   - Flatten
   - Dense(34, relu)
   
   정확도 39%
 
 
  
 2. 과적합 피하기 위해 Dropout 층 추가
    - Conv2D(32, (3,3), relu)
    - MaxPool2D(2,2)
    - Conv2D(32, (3,3), relu)
    - MaxPool2D(2,2)
    - Conv2D(64, (3,3), relu)
    - Flatten
    - Dense(34, relu)
    - **Dropout(0.5)**
    
    
    정확도 41%
    
  유의미한 성능 향상이 보여지진 않았다
 

### try2) 이미지 개수 늘리기

- 이전 훈련 데이터가 3207개 -> `evaluate()` 결과 정확도가 41%
- 훈련 데이터 8345개로 늘리기 -> `evaluate()` 결과 정확도가 76%

# 평가 및 개선점

- 데이터를 기존의 2.6배 정도로 늘렸더니 정확도가 쉽게 개선되었다.
- 세 분류 중 가위를 보자기라고 월등히 잘 못 분류하고 있다
  (가위의 재현율이 매우 낮고 보의 정밀도가 매우 낮다)
- 가위를 더 잘 분류할 수 있게 개선이 필요하다

### 다음에 시도해볼 만한 방법 :  이미지 증강

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
img = load_img(image_dir_path+'/1.jpg')
x = img_to_array(img)  # (3, 150, 150) 크기의 NumPy 배열
x = x.reshape((1,) + x.shape)  # (1, 3, 150, 150) 크기의 NumPy 배열

# 아래 .flow() 함수는 임의 변환된 이미지를 배치 단위로 생성해서
# 지정된 `preview/` 폴더에 저장합니다.
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='paper', save_format='jpeg'):
    i += 1
    if i > 20:
        break 